# Time Series Forecasting Model Tutorial

## Introduction

This tutorial shows **Model Developers** on how to use and configure the **Developer Framework** and the **MRM ValidMind Platform**. The following steps will guide you to **automatically** document and test Time Series Forecasting models: 

- Step 1: Connect Notebook to ValidMind Project 
- Step 2: Import Raw Data
- Step 3: Run Data Validation Test Suite on Raw Data
- Step 4: Preprocess Data
- Step 5: Run Data Validation Test Suite on Processed Data
- Step 6: Load Pre-Trained Models
- Step 7: Run Model Validation Test Suite on Models



## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [ ]:
%pip install --upgrade validmind

## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [ ]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    api_secret="API_SECRET",
    project="xxxxxxxxxxxxxxxxxxxxxxxxx"
)

## Step 1: Connect Notebook to ValidMind Project
Prepare the environment for our analysis by following the **Before you begin** section. First, **import** all necessary libraries and modules required for our analysis. Next, **connect** to the ValidMind MRM platform, which provides a comprehensive suite of tools and services for model validation.

Finally, define and **configure** the specific use case we are working on by setting up any required parameters, data sources, or other settings that will be used throughout the analysis.

#### Explore Test Suites, Test Plans and Tests

In [ ]:
vm.test_suites.list_suites()

In [ ]:
vm.test_plans.list_plans()

In [ ]:
vm.tests.list_tests()

## Step 2: Import Raw Data

#### Import FRED Dataset

In [ ]:
from validmind.datasets.regression import fred as demo_dataset

target_column = demo_dataset.target_column
feature_columns = demo_dataset.feature_columns

df = demo_dataset.load_data()
df.tail(10)

## Step 3: Run Data Validation Test Suite on Raw Data

#### Explore the Time Series Dataset Test Suite

In [ ]:
vm.test_suites.describe_suite("time_series_dataset")

#### Explore Test Plans

In [ ]:
vm.test_plans.describe_plan("time_series_data_quality")

In [ ]:
vm.test_plans.describe_plan("time_series_univariate")

##### Connect Raw Dataset to ValidMind Platform

In [ ]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
)

##### Run Time Series Dataset Test Suite on Raw Dataset

In [ ]:
config={
    
    # TIME SERIES DATA QUALITY PARAMS
    "time_series_outliers": {
        "zscore_threshold": 3,
    },
    "time_series_missing_values":{
        "min_threshold": 2,
    },
    
    # TIME SERIES UNIVARIATE PARAMS 
    "rolling_stats_plot": {
        "window_size": 12    
    },
     "seasonal_decompose": {
        "seasonal_model": 'additive'
    },
     "auto_seasonality": {
        "min_period": 1,
        "max_period": 3
    },
      "auto_stationarity": {
        "max_order": 3,
        "threshold": 0.05
    },
    "auto_ar": {
        "max_ar_order": 2
    },
    "auto_ma": {
        "max_ma_order": 2
    },

    # TIME SERIES MULTIVARIATE PARAMS 
    "lagged_correlation_heatmap": {
        "target_col": demo_dataset.target_column,
        "independent_vars": demo_dataset.feature_columns
    },
    "engle_granger_coint": {
        "threshold": 0.05
    },
}

full_suite = vm.run_test_suite(
    "time_series_dataset",
    dataset=vm_dataset,
    config = config,
)

## Step 4: Preprocess Data

##### Handle Frequencies, Missing Values and Stationairty

In [ ]:
# Sample frequencies to Monthly
resampled_df = df.resample("MS").last()

# Remove all missing values
nona_df = resampled_df.dropna()

# Take the first different across all variables
preprocessed_df = nona_df.diff().dropna()

## Step 5: Run Data Validation Test Suite on Processed Data

In [ ]:
vm_dataset = vm.init_dataset(
    dataset=preprocessed_df,
    target_column=demo_dataset.target_column,
)

full_suite = vm.run_test_suite(
    "time_series_dataset",
    dataset=vm_dataset,
    config = config,
)

## Step 6: Load Pre-Trained Models

#### Load Pre-Trained Models

In [ ]:
from validmind.datasets.regression import fred as demo_dataset
model_A, train_df_A, test_df_A = demo_dataset.load_model('fred_loan_rates_model_3')
model_B, train_df_B, test_df_B = demo_dataset.load_model('fred_loan_rates_model_4')

##### Connect a List of Models To the ValidMind Platform

In [ ]:
# Initialize training and testing datasets for model A
vm_train_ds_A = vm.init_dataset(dataset=train_df_A, type="generic", target_column=demo_dataset.target_column)
vm_test_ds_A = vm.init_dataset(dataset=test_df_A, type="generic", target_column=demo_dataset.target_column)

# Initialize training and testing datasets for model B
vm_train_ds_B = vm.init_dataset(dataset=train_df_B, type="generic", target_column=demo_dataset.target_column)
vm_test_ds_B = vm.init_dataset(dataset=test_df_B, type="generic", target_column=demo_dataset.target_column)

# Initialize model A
vm_model_A = vm.init_model(
    model = model_A, 
    train_ds=vm_train_ds_A, 
    test_ds=vm_test_ds_A)

# Initialize model B
vm_model_B = vm.init_model(
    model = model_B,
    train_ds=vm_train_ds_B,
    test_ds=vm_test_ds_B)


list_of_models = [vm_model_A, vm_model_B]

## Step 7: Run Model Validation Test Suite on Models

#### Explore the Time Series Model Validation Test Suite

In [ ]:
vm.test_suites.describe_test_suite("time_series_model_validation")

#### Explore Test Plans

In [ ]:
vm.test_plans.describe_plan("regression_model_description")

In [ ]:
vm.test_plans.describe_plan("regression_models_evaluation")

In [ ]:
vm.test_plans.describe_plan("time_series_forecast")

In [ ]:
vm.test_plans.describe_plan("time_series_sensitivity")

#### Run Model Validation Test Suite on a List of Models

In [ ]:
config= {
    "regression_forecast_plot_levels": {
        "transformation": "integrate",
    },
    "regression_sensitivity_plot": {
        "transformation": "integrate",
        "shocks": [0.3],
    }
}

full_suite = vm.run_test_suite(
    "time_series_model_validation",
    model = vm_model_B,
    models = list_of_models,
    config = config,
)